In [ ]:
from transformers import AutoModel, AutoTokenizer ,AutoModelForCausalLM
model_TH = "airesearch/wangchanberta-base-att-spm-uncased"


In [2]:
# Load model and tokenizer 
model = AutoModel.from_pretrained(model_TH, 
                                 device_map="cuda", 
                                 torch_dtype="auto", 
                                 trust_remote_code=True,
                                 ) 

tokenizer = AutoTokenizer.from_pretrained(model_TH, use_fast=False)

2025-09-28 23:18:55.379909: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-28 23:18:55.682493: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-09-28 23:18:57.409941: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
`torch_dtype` is deprecated! Use `dtype` instead!


In [3]:
text_th = """
ในสัปดาห์ที่ผ่านมา ตลาดหุ้นไทยปรับตัวขึ้นอย่างต่อเนื่อง โดยได้รับแรงหนุนจากปัจจัยบวกหลายประการ ทั้งการฟื้นตัวของเศรษฐกิจภายในประเทศและการคาดการณ์ว่าธนาคารกลางสหรัฐฯ อาจปรับลดอัตราดอกเบี้ยในอนาคต
นักลงทุนมีความเชื่อมั่นมากขึ้นต่อทิศทางของตลาด ส่งผลให้มีแรงซื้อเข้ามาในหุ้นกลุ่มต่างๆ โดยเฉพาะกลุ่มพลังงานและกลุ่มธนาคาร อย่างไรก็ตาม ผู้เชี่ยวชาญเตือนว่ายังคงต้องติดตามสถานการณ์ความขัดแย้งทางภูมิรัฐศาสตร์ที่อาจส่งผลกระทบต่อความผันผวนของตลาดได้
นอกจากนี้ ภาครัฐยังคงเดินหน้ามาตรการกระตุ้นเศรษฐกิจเพิ่มเติม เพื่อสนับสนุนการบริโภคและการลงทุน โดยคาดว่ามาตรการเหล่านี้จะช่วยเสริมสร้างความแข็งแกร่งให้กับเศรษฐกิจไทยในระยะยาว
"""
# Tokenize the sentence
tokens = tokenizer(text_th, return_tensors='pt')

In [4]:
tokens

{'input_ids': tensor([[    5,    10, 11228,   247,    10,  7953,   109,  2731,   107,    58,
           652,    10, 16778,   611,  3291,    32,  2016,  1867, 13452,    10,
            98, 15279,    16,   283,   306,   136,   703, 15102,     9, 16034,
          1236,  8801, 11997,  1348,  2599,  1590,    10,  4262,   372,  2119,
           429,   100,  2928,    16,   582, 13760, 15781,   238,  5440,  1563,
           171,   208,  3553,   171,  1174,  6269,  1045,  1589,    90,  4443,
          2400, 14445,   946,    83,   649,   497,    63, 13512,  2348,  6198,
            12, 20330,  6390, 21619,    16,   582,    15,  1609,    10,  2622,
           946,  2381,  1297,  2581,   283,   679,  2203,   379,  7031,    13,
          1864,    10, 17073,  1297, 14997, 21264,   954,  3743,   410,   283,
           109,  7560,     6]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [5]:
decode = {tokenizer.decode(word):word for word in tokens['input_ids'][0]}

In [6]:
model = AutoModel.from_pretrained(model_TH)
output = model(**tokens)[0]

In [7]:
output

tensor([[[-0.5875, -1.4182,  0.1009,  ...,  0.1197,  0.2237,  0.1228],
         [-0.3282, -1.8049,  0.1698,  ...,  0.2133,  1.7585,  0.4397],
         [-1.7790, -1.3772,  1.2845,  ..., -0.2261, -1.9724,  0.3860],
         ...,
         [-0.9275, -1.3331, -0.9744,  ...,  1.4161,  0.6069, -2.2569],
         [-0.7577, -0.0811,  0.6786,  ...,  0.8232,  0.1763, -1.2139],
         [-0.5875, -1.4182,  0.1009,  ...,  0.1197,  0.2237,  0.1228]]],
       grad_fn=<NativeLayerNormBackward0>)

In [8]:
from transformers import pipeline
fill_mask = pipeline(task='fill-mask',
         tokenizer=tokenizer,
         model = model_TH,
         revision = 'main',)


Device set to use cuda:0


In [9]:
def fill_mask_pad(input_text):
    return fill_mask(input_text+"<mask>"+'<pad>')

In [10]:
import numpy as np

input_text = "นักเลงคีย์บอร์ดต่อหน้าไม่กล้า" 
for i in range(5):
    word_list= fill_mask_pad(input_text)
    c = np.random.randint(0,5,3)
    input_text += word_list[c[0]]['token_str']
    print(input_text)

นักเลงคีย์บอร์ดต่อหน้าไม่กล้ารับ
นักเลงคีย์บอร์ดต่อหน้าไม่กล้ารับ<unk>
นักเลงคีย์บอร์ดต่อหน้าไม่กล้ารับ<unk>😂
นักเลงคีย์บอร์ดต่อหน้าไม่กล้ารับ<unk>😂🤣
นักเลงคีย์บอร์ดต่อหน้าไม่กล้ารับ<unk>😂🤣🤣


In [11]:
# Using a valid sentiment analysis model for Thai text
classify_multiclass = pipeline(task='sentiment-analysis',
                               tokenizer=tokenizer,
                               model='airesearch/wangchanberta-base-att-spm-uncased',
                               )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [3]:
import pandas as pd
data_train = pd.read_csv('prachatai_train.csv')
data_test = pd.read_csv('prachatai_test.csv')
data_val = pd.read_csv('prachatai_validation.csv')

In [4]:
tags = [
    'politics',
    'human_rights',
    'quality_of_life',
    'international',
    'social',
    'environment',
    'economics',
    'culture',
    'labor',
    'national_security',
    'ict',
    'education',
]

dict_tag_train = {tag:int(data_train[tag].value_counts()[1]) for tag in tags} 
dict_tag_test = {tag:int(data_test[tag].value_counts()[1]) for tag in tags} 
dict_tag_val = {tag:int(data_val[tag].value_counts()[1]) for tag in tags} 

df_count_train = pd.DataFrame(list(dict_tag_train.items()), columns=['tag', 'count'])
df_count_test = pd.DataFrame(list(dict_tag_test.items()), columns=['tag', 'count'])
df_count_val = pd.DataFrame(list(dict_tag_val.items()), columns=['tag', 'count'])

In [39]:
df_count_val

,tag,count
0,politics,3852
1,human_rights,1458
2,quality_of_life,1144
3,international,828
4,social,782
5,environment,764
6,economics,487
7,culture,388
8,labor,375
9,national_security,339


In [5]:
from transformers import AutoModelForSequenceClassification , TrainingArguments, Trainer, DataCollatorWithPadding
import torch
model = AutoModelForSequenceClassification.from_pretrained(
    model_TH,
    num_labels=len(tags),
    problem_type="multi_label_classification"
)

MAX_LEN = 128

from datasets import Dataset

def tokenize_fn(example):
    tokens = tokenizer(example['body_text'], 
                      truncation=True,
                      max_length=MAX_LEN,
                      padding=False)  # Don't pad here, let DataCollator handle it
    # Add labels for multi-label classification
    labels = [float(example[tag]) for tag in tags]
    tokens['labels'] = labels
    return tokens

# Convert pandas DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(data_train)
val_dataset = Dataset.from_pandas(data_val)

# Apply tokenization
tokenized = {
    "train": train_dataset.map(tokenize_fn, remove_columns=train_dataset.column_names),
    "validation": val_dataset.map(tokenize_fn, remove_columns=val_dataset.column_names)
}
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/54379 [00:00<?, ? examples/s]

Map:   0%|          | 0/6721 [00:00<?, ? examples/s]

In [6]:
tokenized["train"].take(2)[1]

{'input_ids': [5,
  10,
  601,
  1003,
  425,
  11236,
  6840,
  641,
  9550,
  2173,
  7130,
  8565,
  10,
  1388,
  14155,
  166,
  26,
  10575,
  336,
  5129,
  7642,
  7801,
  83,
  1370,
  947,
  32,
  605,
  480,
  357,
  10,
  76,
  162,
  11,
  10,
  2425,
  5210,
  3272,
  10496,
  275,
  2982,
  425,
  5221,
  734,
  472,
  947,
  636,
  1984,
  641,
  985,
  497,
  3548,
  2633,
  17663,
  16,
  21727,
  10,
  12,
  622,
  1304,
  12430,
  35,
  600,
  501,
  112,
  513,
  8565,
  952,
  2643,
  746,
  10,
  8519,
  7515,
  2190,
  10,
  579,
  9130,
  2124,
  4526,
  7182,
  351,
  8565,
  3925,
  22,
  10,
  409,
  10,
  1123,
  10,
  3478,
  97,
  668,
  2246,
  10,
  460,
  377,
  10,
  65,
  10,
  168,
  222,
  622,
  702,
  980,
  567,
  968,
  760,
  8565,
  308,
  45,
  10,
  4435,
  8953,
  8565,
  2499,
  1898,
  4096,
  10,
  5098,
  7443,
  2212,
  46,
  289,
  10,
  22560,
  32,
  760,
  640,
  6],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [7]:
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
sigmoid = torch.nn.Sigmoid()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert to numpy arrays if they aren't already
    if isinstance(logits, torch.Tensor):
        logits = logits.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.detach().cpu().numpy()
    
    # Apply sigmoid to get probabilities
    probs = 1 / (1 + np.exp(-logits))  # sigmoid function
    # Convert to binary predictions
    preds = (probs >= 0.5).astype(int)
    y_true = labels.astype(int)

    return {
        "f1_micro": f1_score(y_true, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(y_true, preds, average="macro", zero_division=0),
        "precision_micro": precision_score(y_true, preds, average="micro", zero_division=0),
        "recall_micro": recall_score(y_true, preds, average="micro", zero_division=0)
    }


In [8]:

OUTPUT_DIR = "wcberta-prachathai67k"
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    eval_strategy ="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none",
    save_total_limit=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f"{OUTPUT_DIR}/logs",
    remove_unused_columns=True
   )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_8379/3818179347.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 6, 'bos_token_id': 5}.


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Precision Micro,Recall Micro
1,0.179500,0.171885,0.728551,0.648837,0.782390,0.681644
2,0.150600,0.165269,0.744290,0.669600,0.799204,0.696438
3,0.134200,0.165133,0.756356,0.687707,0.793363,0.722648


TrainOutput(global_step=40785, training_loss=0.16829485802640515, metrics={'train_runtime': 24469.2873, 'train_samples_per_second': 6.667, 'train_steps_per_second': 1.667, 'total_flos': 1.073175053363712e+16, 'train_loss': 0.16829485802640515, 'epoch': 3.0})

In [12]:
test = Dataset.from_pandas(data_test)
tokenized_test = test.map(tokenize_fn, remove_columns=test.column_names)

Map:   0%|          | 0/6789 [00:00<?, ? examples/s]

In [15]:
tokenized_test.take(2)[1]

{'input_ids': [5,
  2117,
  10,
  40,
  11,
  40,
  11,
  10,
  569,
  312,
  972,
  271,
  54,
  231,
  2763,
  792,
  1171,
  21,
  20573,
  24,
  12784,
  10,
  676,
  16,
  9739,
  10934,
  1332,
  554,
  1242,
  19,
  20549,
  1357,
  1579,
  10,
  3,
  6974,
  18,
  222,
  19,
  10622,
  327,
  17624,
  8159,
  18,
  10,
  6762,
  1802,
  7862,
  2328,
  14137,
  14262,
  108,
  1065,
  622,
  10,
  17318,
  9,
  9563,
  10,
  396,
  56,
  9,
  10300,
  10,
  3835,
  979,
  50,
  755,
  350,
  2970,
  13,
  676,
  4033,
  10,
  16,
  9739,
  10934,
  1332,
  599,
  273,
  2216,
  32,
  2843,
  79,
  19,
  3,
  3234,
  21433,
  1579,
  10,
  3,
  6974,
  18,
  10,
  9388,
  1351,
  9401,
  12,
  215,
  144,
  569,
  584,
  713,
  231,
  271,
  2407,
  379,
  1071,
  10,
  3748,
  14137,
  14262,
  108,
  31,
  287,
  1522,
  673,
  2117,
  10,
  40,
  11,
  40,
  11,
  10,
  559,
  10,
  3082,
  6],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [40]:
test_data = data_test.loc[1:2,['body_text']+tags]
test_data = Dataset.from_pandas(test_data)
test_data_tok = test_data.map(tokenize_fn, remove_columns=test_data.column_names)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [68]:
test_data.take(2)[1]

{'body_text': 'สมาคมเพื่อสันติภาพ นำเครือข่ายเด็กและเยาวชน ภาคประชาสังคม 15 องค์กรร่วมเดินรณรงค์ยุติความรุนแรง สร้างพื้นที่ปลอดภัยเนื่องในเด็กแห่งชาติที่ปัตตานี\n\n\n\nเมื่อวันที่ 8 มกราคม 2559 สมาคมเพื่อสันติภาพชายแดนใต้และองค์กรภาคีเครือข่ายจัดกิจกรรม “พื้นที่ปลอดภัย หยุดความรุนแรงต่อเด็ก” เนื่องในงานวันเด็กแห่งชาติปี 2559 โดยการเดินรณรงค์ยุติความรุนแรงต่อเด็กและเยาวชนจากวิทยาลัยเทคนิคปัตตานี ผ่านถนนเจริญประดิษฐ์ เข้าสู่หอประชุมสำนักงานอธิการบดี มหาวิทยาลัยสงขลานครินทร์ (ม.อ.) วิทยาเขตปัตตานีเพื่ออ่านแถลงการณ์ของแต่ละองค์กร มีผู้เข้าร่วมประมาณ 500 คน\nโดยเครือข่ายเด็กและเยาวชนในเขตพื้นที่เทศบาลเมืองปัตตานี รวมถึงองค์กรภาคประชาสังคมที่เข้าร่วม15 องค์กร ประกอบด้วย1.สมาคมเพื่อสันติภาพชายแดนใต้2.ศูนย์เฝ้าระวังสถานการณ์ภาคใต้3.สถาบันสมุทรรัฐเอเชียตะวันออกเฉียงใต้ศึกษา ม.อ.ปัตตานี4.ชมรมนักศึกษารักสันติ5.ชมรม ม.อ.ทำดีเพื่อสังคม6.ชุมนุมสิงห์สันติภาพ 7.โรงเรียนเบญจมราชูทิศปัตตานี\n8.โรงเรียนเดชะปัตตนยานุกูล9.วิทยาลัยเทคนิคปัตตานี10.วิทยาลัยอาชีวศึกษาปัตตานี11.วิทยาลัยการอาชีพปัตตานี12.วิทยาลั

In [65]:
pre_result = trainer.predict(test_data_tok.select(range(2)))
topic = [j for i,j in enumerate(tags) if pre_result.label_ids[1][i] == 1]
topic

['politics', 'social', 'national_security']

In [70]:
import json, os

run_dir = "wcberta-prachathai67k/checkpoint-27190"  # โฟลเดอร์รันหลักที่มี trainer_state.json
with open(os.path.join(run_dir, "trainer_state.json"), encoding="utf-8") as f:
    st = json.load(f)

print("metric_for_best_model:", st.get("metric_for_best_model"))
print("best_metric:", st.get("best_metric"))
print("best_model_checkpoint:", st.get("best_model_checkpoint"))

metric_for_best_model: None
best_metric: 0.7442904548116338
best_model_checkpoint: wcberta-prachathai67k/checkpoint-27190


In [71]:
import json, os

run_dir = "wcberta-prachathai67k/checkpoint-40785"  # โฟลเดอร์รันหลักที่มี trainer_state.json
with open(os.path.join(run_dir, "trainer_state.json"), encoding="utf-8") as f:
    st = json.load(f)

print("metric_for_best_model:", st.get("metric_for_best_model"))
print("best_metric:", st.get("best_metric"))
print("best_model_checkpoint:", st.get("best_model_checkpoint"))

metric_for_best_model: None
best_metric: 0.7563563372204167
best_model_checkpoint: wcberta-prachathai67k/checkpoint-40785


In [72]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

best_ckpt = "wcberta-prachathai67k/checkpoint-40785"
export_dir = "wcberta-prachathai67k-best"

model = AutoModelForSequenceClassification.from_pretrained(best_ckpt)
tok   = AutoTokenizer.from_pretrained(best_ckpt)

model.save_pretrained(export_dir)
tok.save_pretrained(export_dir)

# (ถ้าคุณมีไฟล์ mapping ชื่อคลาส)
import json
LABEL_NAMES = ["politics","human_rights","quality_of_life","international","social",
               "environment","economics","culture","labor","national_security","ict","education"]
with open(f"{export_dir}/label_names.json","w",encoding="utf-8") as f:
    json.dump(LABEL_NAMES, f, ensure_ascii=False, indent=2)